In [4]:
# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Vamos usar o pandas para uma tabela bonita
import pandas as pd

# Vamos fazer o gráfico das funções, por isso precisamos usar o módulo matplotlib.
# Vamos também escolher o modo de exibição do gráfico. No nosso caso, para simplificar, escolhemos
%matplotlib inline
# Em seguida, importamos apenas o submódulo pyplot da matplotlib com o "apelido" plt:
import matplotlib.pyplot as plt

# Por fim, importamos algumas facilidades para a exibição de LaTeX e outras saídas
from IPython.display import display, clear_output, Markdown, Latex

# Quantos minutos?

Na Cidade de Tecnópolis, existem duas operadoras de telefonia celular. "MundoFone" oferece um plano mensal com uma tarifa de R\\$35 por 120 minutos, mais R\\$0,10 por cada minuto adicional. "UniverCell" oferece um plano mensal com uma tarifa de R\\$30 por 100 minutos, e cada minuto adicional custa R\\$0,15 até um máximo de 300 minutos. Depois disso, cada minuto extra custa R\\$0,05. 

Quantos minutos você deveria utilizar por mês para que "MundoFone" seja a melhor opção, isto é, custe menos que "UniverCell"?

## Preparando o problema (inspeção)

Primeiramente, vamos olhar para como calcular a franquia mensal para cada operadora. Normalmente, quando olhamos para problemas assim, escolhemos alguns valores ao acaso e calculamos o resultado. Vamos tentar uma tabela:

In [1]:
def mundofone(minutos):
    if minutos <= 120:
        custo = 35
    else:
        custo = 35+(minutos-120)*0.1
    return custo

In [5]:
def univercell(minutos):
    if minutos <= 100:
        custo = 30
    elif minutos <= 300:
        custo = 30 + (minutos-100)*0.15
    else:
        custo = 30 + 200*0.15 + (minutos-300)*0.05
    return custo

In [6]:
minutos_input = widgets.Text(
                    value='',
                    placeholder='Entre minutos (decimal)',
                    description="Minutos =",
                )

# Inicializar tabela com a primeira linha
dados = [['Tarifa Básica', 35, 30]] 
tabela = pd.DataFrame(dados, columns = ['Minutos', 'MundoFone', 'UniverCell']) 

interact_manual.opts['manual_name'] = 'Calcular Custo'

@interact_manual(minutos=minutos_input, tabela=fixed(tabela))
def atualizar_tabela(minutos, tabela):
    if minutos != '':
        tempo = float(minutos)
        if len(tabela[tabela['Minutos'] == tempo]) > 0:
            # este valor já foi calculado; não faça nada!
            pass
        else:
            tabela.loc[len(tabela)+1] = (tempo, mundofone(tempo), univercell(tempo))
    # eliminar o índice para simplificar a tabela
    tabela.index = ['']*len(tabela)
    display(tabela)

interactive(children=(Text(value='', description='Minutos =', placeholder='Entre minutos (decimal)'), Button(d…

Agora, seria interessante ver como esses valores sem comparam em um gráfico. Vamos marcar, os valores calculados acima da seguinte forma: o tempo será marcado no eixo horizontal, e o custo de cada companhia será marcado no eixo vertical.

In [5]:
fig, ax = plt.subplots()
plt.close(fig)

botao_pontos = widgets.Button(
                    description='Mostrar valores calculados no gráfico',
                    tooltip='Mostrar valores calculados no gráfico',
                    layout=widgets.Layout(width='auto')
                    )

botao_conectar = widgets.Button(
                    description='Conectar pontos',
                    tooltip='Conectar pontos',
                    layout=widgets.Layout(width='auto')
                    )

display(widgets.HBox([botao_pontos, botao_conectar]))
saida_graficos = widgets.Output()
display(widgets.HBox(children=(saida_graficos,)))

def graficar_pontos(grafico):
    ax.clear()
    ax.plot(tabela['Minutos'][1:], tabela['MundoFone'][1:], 'r*', label="MundoFone")
    ax.plot(tabela['Minutos'][1:], tabela['UniverCell'][1:], 'bo', label="UniverCell")
    ax.legend()
    with saida_graficos:
        clear_output(wait=True)
        display(fig)
        
def conectar_pontos(conectar):
    ax.clear()
    # Vamos organizar os valores de minutos na tabela para que
    # nosso gráfico fique organizado
    entradas = tabela[1:].sort_values(by='Minutos')
    ax.plot(entradas['Minutos'], entradas['MundoFone'], 'r*')
    ax.plot(entradas['Minutos'], entradas['UniverCell'], 'bo')
    ax.plot(entradas['Minutos'], entradas['MundoFone'], 'r')
    ax.plot(entradas['Minutos'], entradas['UniverCell'], 'b')
    ax.legend(("MundoFone", "UniverCell"))
    with saida_graficos:
        clear_output(wait=True)
        display(fig)

botao_pontos.on_click(graficar_pontos)
botao_conectar.on_click(conectar_pontos)

Agora, podemos ver que existe um intervalo em que os pontos (e a linha) vermelhos ficam abaixo dos pontos azuis; isso significa que MundoFone é mais barata que UniverCell somente nesse intervalo. Vamos ver quando isso ocorre.

Do gráfico, podemos ver que o intervalo em que estamos interessados é determinado pelos dois pontos em que as linhas azul e vermelha se cruzam. (Se você não consegue ver esses dois pontos de intersecção, tente adicionar mais valores à sua tabela; digamos, valores até 500 minutos.) Agora, como podemos encontrar estes pontos?

## Encontrando a intersecção das duas linhas

Ambas linhas são determinadas por *funções*, que são maneiras de conectar os minutos aos custos correspondentes em cada companhia de telefonia. Apenas para simplificar, vamos dar um apelido ao valor do custo mensal de cada companhia. O custo para MundoFone será chamado $W$ e o custo para UniverCell será chamado $U$. 

Podemos ver que ambos os custos são determinados por quantos minutos são usados em cada mês. Podemos então dizer coisas assim:

In [6]:
tabela.set_index('Minutos')
for indice, linha in tabela[1:].iterrows():
    print('Para {0} minutos, temos W({0}) = {1}, U({0}) = {2}.'.format(int(linha[0]), linha[1], linha[2]))

Note que para valores que não estão na tabela, não podemos decidir diretamente quanto isso custaria, a não ser que calculemos este número também. Felizmente, existe uma maneira mais fácil de descrever o custo total sem termos que listar todos as possíveis quantidades de minutos: usando a *lei* da função. Em nosso caso, tivemos o seguinte: vamos chamar $t$ a quantidade total de minutos usada naquele mês. Então,

In [7]:
# Título
display(Markdown('**Para MundoFone:**<br/> *(Pressione Enter após entrar com o valor)*'))

# Mostrar nome da função - W(t)
nome_mundofone = widgets.Output(layout=widgets.Layout(min_width='80px'))
with nome_mundofone:
    display(Latex('$W(t)= $'))
    
# Mostrar primeiro intervalo para a definição da função W
condicao1_mundofone = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao1_mundofone'))
with condicao1_mundofone:
    display(Latex('para $t \leq 120$'))
    
# Mostrar segundo intervalo para a definição da função W
condicao2_mundofone = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao2_mundofone'))
with condicao2_mundofone:
    display(Latex('para $t > 120$.'))

# Mostrar gráfico resultante
fig_mundofone, ax_mundofone = plt.subplots()
grafico_mundofone = widgets.Output()
plt.close(fig_mundofone)

def preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone):
    with grafico_mundofone:
        entradas = [0, 120, 500]
        saidas_mundofone = []
        for t in entradas:
            saidas_mundofone.append(mundofone(t))
        ax_mundofone.plot(entradas, saidas_mundofone, 'r', label='MundoFone')
        ax_mundofone.legend()
        display(fig_mundofone)

preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone)
        
# Widgets para o input das leis da função
caixa1_mundofone = widgets.Text(layout=widgets.Layout(width='150px', grid_area='caixa1_mundofone'))
caixa2_mundofone = widgets.Text(layout=widgets.Layout(width='150px', grid_area='caixa2_mundofone'))

# Widgets são mostrados em uma tabela
grid_mundofone = widgets.GridBox(
    children=[caixa1_mundofone, condicao1_mundofone, 
              caixa2_mundofone, condicao2_mundofone],
    layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
                "caixa1_mundofone condicao1_mundofone"
                "caixa2_mundofone condicao2_mundofone"
                ''')
    )

# Este widget contém os três widgets definidos acima, lado a lado.
caixa_horizontal = widgets.HBox(
    children=[nome_mundofone, grid_mundofone, grafico_mundofone],
    layout=widgets.Layout(width='90%', align_items='center', justify_content='center')
    )

# As funções abaixo desenham o gráfico das condições informadas pelos usuários 
# nas caixas de texto definidas acima. Os gráficos são então comparados com 
# o gráfico verdadeiro da função, para que se verifique que a lei encontrada 
# representa as condições que desejamos para essa função.
def grafico_mundofone_condicao1(caixa):
    entradas = [0, 120]
    valores = []
    for t in entradas:
        # atenção com eval!
        valores.append(eval(caixa.value))
    ax_mundofone.plot(entradas, valores, 'k-', label="Meu gráfico ($0\leq t\leq 120$)")
    ax_mundofone.legend()
    with grafico_mundofone:
        clear_output(wait=True)
        display(fig_mundofone)
        
def grafico_mundofone_condicao2(caixa):
    entradas = [120, 500]
    valores = []
    for t in entradas:
        valores.append(eval(caixa.value))
    ax_mundofone.plot(entradas, valores, 'k-', label="Meu gráfico ($t>120$)")
    ax_mundofone.legend()
    with grafico_mundofone:
        clear_output(wait=True)
        display(fig_mundofone)
    
# Finalmente, mostramos todos os widgets
display(caixa_horizontal)
caixa1_mundofone.on_submit(grafico_mundofone_condicao1)
caixa2_mundofone.on_submit(grafico_mundofone_condicao2)

# Definimos um botão para limpar os gráficos caso um erro seja cometido.
botao_limpar_mundofone = widgets.Button(
                            description='Limpar gráfico',
                            tooltip='Limpar gráfico',
                            layout=widgets.Layout(width='auto')
                            )

def limpar_grafico_mundofone(botao):
    with grafico_mundofone:
        clear_output(wait=True)
        ax_mundofone.clear()
        preparar_fig_mundofone(fig_mundofone, ax_mundofone, grafico_mundofone)

display(botao_limpar_mundofone)
botao_limpar_mundofone.on_click(limpar_grafico_mundofone)

**Para MundoFone:**<br/> *(Pressione Enter após entrar com o valor)*

Button(description='Limpar gráfico', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Limpar gráfico…

Vamos repetir para UniverCell (lembrando que aqui temos 3 faixas de preço):

In [8]:
# Título
display(Markdown('**Para UniverCell:**<br/> *(Pressione enter após entrar com o valor)*'))

# Mostrar nome da função - U(t)
nome_univercell = widgets.Output(layout=widgets.Layout(min_width='80px'))
with nome_univercell:
    display(Latex('$U(t)= $'))
    
# Mostrar primeiro intervalo de definição da função U
condicao1_univercell = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao1_univercell'))
with condicao1_univercell:
    display(Latex('para $t \leq 100$'))
    
# Mostrar segundo intervalo de definição da função U
condicao2_univercell = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao2_univercell'))
with condicao2_univercell:
    display(Latex('para $100 < t \leq 300$'))

# Mostrar terceiro intevalo de definição da função U
condicao3_univercell = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condicao3_univercell'))
with condicao3_univercell:
    display(Latex('para $t > 300$.'))

# Mostrar gráfico resultante
fig_univercell, ax_univercell = plt.subplots()
plt.close(fig_univercell)
grafico_univercell = widgets.Output()

def preparar_fig_univercell(fig_univercell, ax_univercell, grafico_univercell):
    with grafico_univercell:
        entradas = [0, 100, 300, 500]
        valores = []
        for t in entradas:
            valores.append(univercell(t))
        ax_univercell.plot(entradas, valores, 'b', label='UniverCell')
        ax_univercell.legend()
        display(fig_univercell)

preparar_fig_univercell(fig_univercell, ax_univercell, grafico_univercell)
    
# Widgets para o input das leis da função
caixa1_univercell = widgets.Text(layout=widgets.Layout(width='auto', grid_area='caixa1_univercell'))
caixa2_univercell = widgets.Text(layout=widgets.Layout(width='auto', grid_area='caixa2_univercell'))
caixa3_univercell = widgets.Text(layout=widgets.Layout(width='auto', grid_area='caixa3_univercell'))

# Widgets são mostrados em uma tabela
grid_univercell = widgets.GridBox(
    children=[caixa1_univercell, condicao1_univercell, 
              caixa2_univercell, condicao2_univercell, 
              caixa3_univercell, condicao3_univercell],
    layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
                "caixa1_univercell condicao1_univercell"
                "caixa2_univercell condicao2_univercell"
                "caixa3_univercell condicao3_univercell"
                ''')
    )

# Este widget contém os três widgets definidos acima, lado a lado.
caixa_horizontal = widgets.HBox(children=[nome_univercell, grid_univercell, grafico_univercell],
                    layout=widgets.Layout(width='100%', align_items='center', justify_content='center'))

# As funções abaixo desenham o gráfico das condições informadas pelos usuários 
# nas caixas de texto definidas acima. Os gráficos são então comparados com 
# o gráfico verdadeiro da função, para que se verifique que a lei encontrada 
# representa as condições que desejamos para essa função.
def grafico_univercell_condicao1(caixa):
    entradas = [0, 100]
    valores = []
    for t in entradas:
        valores.append(eval(caixa.value))
    ax_univercell.plot(entradas, valores, 'k-', label="Meu gráfico ($0\leq t\leq 100$)")
    ax_univercell.legend()
    with grafico_univercell:
        clear_output(wait=True)
        display(fig_univercell)
        
def grafico_univercell_condicao2(caixa):
    entradas = [100, 300]
    valores = []
    for t in entradas:
        valores.append(eval(caixa.value))
    ax_univercell.plot(entradas, valores, 'k-', label="Meu gráfico ($100<t\leq 300$)")
    ax_univercell.legend()
    with grafico_univercell:
        clear_output(wait=True)
        display(fig_univercell)

def grafico_univercell_condicao3(caixa):
    entradas = [300, 500]
    valores = []
    for t in entradas:
        valores.append(eval(caixa.value))
    ax_univercell.plot(entradas, valores, 'k-', label="Meu gráfico ($t > 500$)")
    ax_univercell.legend()
    with grafico_univercell:
        clear_output(wait=True)
        display(fig_univercell)

# Finalmente, mostramos todos os widgets
display(caixa_horizontal)
caixa1_univercell.on_submit(grafico_univercell_condicao1)
caixa2_univercell.on_submit(grafico_univercell_condicao2)
caixa3_univercell.on_submit(grafico_univercell_condicao3)

# Definimos um botão para limpar os gráficos caso um erro seja cometido.
botao_limpar_univercell = widgets.Button(
                description='Limpar gráfico',
                tooltip='Limpar gráfico',
                layout=widgets.Layout(width='auto')
                )

def limpar_grafico_univercell(botao):
    with grafico_univercell:
        clear_output(wait=True)
        ax_univercell.clear()
        preparar_fig_univercell(fig_univercell, ax_univercell, grafico_univercell)

display(botao_limpar_univercell)
botao_limpar_univercell.on_click(limpar_grafico_univercell)

**Para UniverCell:**<br/> *(Pressione enter após entrar com o valor)*

Button(description='Limpar gráfico', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Limpar gráfico…

Agora, vemos que as leis para $W(t)$ e $U(t)$ são

In [9]:
mostrar_w = widgets.Output()
mostrar_v = widgets.Output()
with mostrar_w:
    display(Latex(r'$$W(t) = \begin{cases} 35, & t \leq 120\\ 35+0.1(t-120), & t> 120\end{cases}$$'))
with mostrar_v:
    display(Latex(r'$$U(t) = \begin{cases} 30, & t\leq 100\\30+0.15(t-100), &100<t\leq 300\\ 30 + 200(0.15) + 0.05(t-300), & t>300.\end{cases}$$'))

# Vamos usar um widget de "acordeão", que mantém os seus conteúdos escondidos até que se clique neles,
# para evitar "spoilers" :)
# A opção "selected_index=None" nos garante que nenhum dos dois resultados será mostrado
# antes que o usuário clique naquela opção.
accordion = widgets.Accordion(children=[mostrar_w, mostrar_v], selected_index=None)
accordion.set_title(0, 'MundoFone:')
accordion.set_title(1, 'UniverCell:')
display(accordion)

Accordion(children=(Output(), Output()), selected_index=None, _titles={'0': 'MundoFone:', '1': 'UniverCell:'})

Em Python, representamos isso da seguinte forma: para MundoFone, temos 

In [10]:
def W(t):
    if t <= 120:
        W = 35
    else:
        W = 35+(t-120)*0.1
    return W

e para UniverCell, temos  

In [11]:
def U(t):
    if t <= 100:
        U = 30
    elif t <= 300:
        U = 30 + (t-100)*0.15
    else:
        U = 30 + 200*0.15 + (t-300)*0.05
    return U

Finalmente, para encontrarmos os pontos de intersecção entre as duas curvas, podemos ver que para o ponto mais à esquerda, estamos no intervalo em que $100<t<300$, e assim usaremos a segunda lei para $W$ e a segunda lei para $U$:

\begin{align*}
    35+0.1(t-120) &= 30+0.15(t-100)\\
    35 + 0.1t - 12 &= 30+0.15t - 15\\
    0.1t-0.15t &= 30-15-35+12\\
    -0.05t &= -8
\end{align*}
e assim,
$$t = \frac{800}{5} = 160.$$

De fato, podemos verificar que $W(160)$ é

In [12]:
W(160)

39.0

Assim, $U(160)$ é

In [13]:
U(160)

39.0

Em seguida, temos um ponto onde $400<t<500$, e assim devemos ter

\begin{align*}
    35+0.1(t-120) &= 30 + 200(0.15) + 0.05(t-300)\\
    35 + 0.1t - 12 &= 30 + 30 + 0.05t - 15\\
    0.1t-0.05t &= 30+30-15-35+12\\
    0.05t &= 22
\end{align*}
e assim,
$$t = \frac{2200}{5} = 440.$$

De fato, 

In [14]:
W(440)

67.0

e

In [15]:
U(440)

67.0

## Resposta

Para que MundoFone seja a melhor opção, nosso uso mensal deve ficar entre 160 e 440 minutos.